In [1]:
import tellurium as te # Python-based modeling environment for kinetic models
import roadrunner as rr # High-performance simulation and analysis library
import numpy as np # Scientific computing package
import matplotlib.pylab as plt # Additional Python plotting utilities
import pandas as pd
import os
import seaborn as sns

In [2]:
# 2 ligand 1 integrin  
# ligands: fibronectin and von Willebrand Factor A (initial values adapted from kidney orgaoid iBAQ values)
# integrin avB3 (initial value from Hudson et al)
# here the rate for ligand binding-unbinding  is from Hudson et al (both ligands) 
# activation/inactivation is from  Yu et al. 2017
# clustering forward is numerically from Hudson, then reverse is estimated using the Kd from Yu et al .
Ant_str = """
  model test # activation model 
  species i, I, $F, IF, $W, IW, C; 
  #inactive integrin, active integrin, fibronectin, integrin+fibronectin, vonWillebrand Factor A, integrin+vonWillebrand Factor A, clustered integrins respectively.
  
  #set initial values:
  i = 0.05; # integrin avB3
  I = 0; 
  F = 0.18   ; #fibronectin
  IF = 0;
  W = 0.33   ; #von Willebrand factor A
  IW = 0;  
  C = 0;
  J1: i -> I; k1*i - k2*I; # reaction; reaction rate law;   # activation step, k1 rate of activation, k2 rate of inactivation
  J2: I + $F -> IF; k3*I*F - k4*IF;                          # ligand binding step, k3 rate of fibronectin binding, k4 rate of dissociation
  J3: I + $W -> IW; k5*I*W - k6*IW;                          # alternative ligand binding step, k5 rate of vWFA binding, k6 rate of dissociation
  J4: IF + IF -> C; k7*IF^2 - k8*C;
  J5: IW + IW -> C; k7*IW^2 - k8*C;
  J6: IF + IW -> C; k7*IF*IW - k8*C;                         # clustering step, k7 rate of clustering, k8 rate of dissociation
  k1 = 5*10^6; k2 = 10^8; ; k3 = 1.6*10^8 ; k4 = 3.5*10^-1; k5 = 1.6*10^4 ; k6 = 2.3*10^-2; k7 = 1.6*10^8; k8 = 0.5*10^7; # assign constant values to global parameters
  end
  """
r2 = te.loada(Ant_str)

print(te.getODEsFromModel(r2))


vJ1 = k1*i-k2*I
vJ2 = k3*I*F-k4*IF
vJ3 = k5*I*W-k6*IW
vJ4 = k7*pow(IF,2)-k8*C
vJ5 = k7*pow(IW,2)-k8*C
vJ6 = k7*IF*IW-k8*C

di/dt = -vJ1
dI/dt = vJ1 - vJ2 - vJ3
dIF/dt = vJ2 - 2.0*vJ4 - vJ6
dIW/dt = vJ3 - 2.0*vJ5 - vJ6
dC/dt = vJ4 + vJ5 + vJ6



In [ ]:
# I rearranged auto-generated equations.
di/dt = - k1*I + k2*I 
dI/dt = k1*I - k2*I - k3*I*F + k4*IF - k5*I*W + k6*IW 
dIF/dt = k3*I*F - k4*IF - 2*k7*IF^2 + 3*k8*C – k7*IF*IW 
dIW/dt = k5*I*W - k6*IW - 2*k7*IW^2 + 3*k8*C - k7*IF*IW 
dC/dt = k7*IF^2 + k7*IW^2 + k7*IF*IW – 3*k8*C

 
